In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def output_results(out):
    pd.DataFrame({'deaths': out, 'Id': range(11)}).to_csv('mewmew_preds.csv', index=False)
        
def calculate_error(pred, real):
    return np.sqrt(np.mean((pred - real) ** 2))

training_df = pd.read_csv('phase1_training_data.csv')
training_df['date'] = training_df['date'].astype('datetime64')
training_df['country_id'].fillna('Unknown', inplace=True)

for col in training_df.columns[2:]:
    training_df[col+'_daily'] = training_df.groupby('country_id')[col].transform(lambda x: x.diff().fillna(0))
    
# ca_train_df = training_df[(training_df['country_id'] == 'CA') & (training_df['date'] >= '2020-03-15')]
ca_train_df = training_df[training_df['country_id'] == 'CA']

second_val_df = pd.read_csv('general_phase_1_answer.csv')
ca_val_df = second_val_df[second_val_df['country_id'] == 'CA']

In [2]:
from AutoReg_MultiVar import AutoRegressionModelMultiVar as armv

errs = []

i = 0
for begining_ind in range(70, 151):
    data = ca_train_df[begining_ind:]
    deaths = data['deaths'].values
    cases = data['cases'].values
    for k_lag in range(1, 25):
        for seq_case_count in range(6):
            for case_lag in range(11, 26):
                lag_indices = list(np.arange(case_lag, min(case_lag+seq_case_count, 26)))
                model = armv(k=k_lag, bias=True)
                if len(lag_indices) == 0:
                    model.fit(deaths, np.empty(0), np.empty(0))
                else:
                    model.fit(deaths, [cases], [lag_indices])
                error = calculate_error(ca_val_df['deaths'][:11].values, model.predict(len(data), len(data)+10))
                errs.append((begining_ind, k_lag, seq_case_count, case_lag, error))
                i += 1
                if i % 1000 == 0:
                    print("i is:", i)

i is: 1000
i is: 2000
i is: 3000
i is: 4000
i is: 5000
i is: 6000
i is: 7000
i is: 8000
i is: 9000
i is: 10000
i is: 11000
i is: 12000
i is: 13000
i is: 14000
i is: 15000
i is: 16000
i is: 17000
i is: 18000
i is: 19000
i is: 20000
i is: 21000
i is: 22000
i is: 23000
i is: 24000
i is: 25000
i is: 26000
i is: 27000
i is: 28000
i is: 29000
i is: 30000
i is: 31000
i is: 32000
i is: 33000
i is: 34000
i is: 35000
i is: 36000
i is: 37000
i is: 38000
i is: 39000
i is: 40000
i is: 41000
i is: 42000
i is: 43000
i is: 44000
i is: 45000
i is: 46000
i is: 47000
i is: 48000
i is: 49000
i is: 50000
i is: 51000
i is: 52000
i is: 53000
i is: 54000
i is: 55000
i is: 56000
i is: 57000
i is: 58000
i is: 59000
i is: 60000
i is: 61000
i is: 62000
i is: 63000
i is: 64000
i is: 65000
i is: 66000
i is: 67000
i is: 68000
i is: 69000
i is: 70000
i is: 71000
i is: 72000
i is: 73000
i is: 74000
i is: 75000
i is: 76000
i is: 77000
i is: 78000
i is: 79000
i is: 80000
i is: 81000
i is: 82000
i is: 83000
i is: 84000
i

In [7]:
error_df = pd.DataFrame(errs, columns=['begining_index', 'k_lag', 'case_sequence_count', 'case_lag', 'error'])
error_df

,begining_index,k_lag,case_sequence_count,case_lag,error
0,70,1,0,11,49.742718
1,70,1,0,12,49.742718
2,70,1,0,13,49.742718
3,70,1,0,14,49.742718
4,70,1,0,15,49.742718
...,...,...,...,...,...
174955,150,24,5,21,18.228198
174956,150,24,5,22,15.832143
174957,150,24,5,23,15.446387
174958,150,24,5,24,16.318384


In [138]:
error_df.to_csv('errors_1.csv', index=False)

In [10]:
error_df.sort_values('error')

,begining_index,k_lag,case_sequence_count,case_lag,error
26050,82,2,2,21,6.296701
161343,144,17,4,14,7.327147
84573,109,4,4,14,7.466346
88803,111,3,4,14,7.475531
103923,118,3,4,14,7.507800
...,...,...,...,...,...
23,70,1,1,19,443.294345
2181,71,1,1,17,443.585977
22,70,1,1,18,449.755527
20,70,1,1,16,450.966107


In [20]:
data = ca_train_df[begining_ind:]
deaths = data['deaths'].values
cases = data['cases'].values

model = armv(k=2, bias=True)
lag_indices = list(np.arange(case_lag, min(case_lag+seq_case_count, 26)))
model.fit(data, )

6.296700513917712

In [21]:
output_results(model.predict(len(data), len(data)+10))

In [120]:
new_training_df = pd.read_csv('general_phase_1_answer.csv')
new_training_df['country_id'].fillna('Unknown', inplace=True)
new_training_df = new_training_df[new_training_df['country_id'] == 'CA']

new_ca_train_df = pd.concat((ca_train_df, new_training_df[:-5]))

In [125]:
test_errors = []
models = []
for ind, row in error_df[error_df['error'] <= 10].iterrows():
    data = new_ca_train_df[int(row.begining_index):]
    deaths = data['deaths'].values
    cases = data['cases'].values
    model = armv(k=int(row.k_lag), bias=True)
    lag_indices = list(np.arange(row.case_lag, min(row.case_lag+row.case_sequence_count, 26)).astype('int'))
    model.fit(deaths, [cases], [lag_indices])
    error = calculate_error(ca_val_df['deaths'][-5:].values, model.predict(len(data), len(data)+4))
    test_errors.append((row.begining_index, row.k_lag, row.case_sequence_count, row.case_lag, error))

In [126]:
test_error_df = pd.DataFrame(test_errors, columns=['begining_index', 'k_lag', 'case_sequence_count', 'case_lag', 'error'])
test_error_df['error'].mean()

15.927593503983317

In [131]:
preds = []
models = []
for ind, row in error_df[error_df['error'] <= 15].iterrows():
    data = new_ca_train_df[int(row.begining_index):]
    deaths = data['deaths'].values
    cases = data['cases'].values
    model = armv(k=int(row.k_lag), bias=True)
    lag_indices = list(np.arange(row.case_lag, min(row.case_lag+row.case_sequence_count, 26)).astype('int'))
    model.fit(deaths, [cases], [lag_indices])
    preds.append([model.predict(len(data), len(data)+4)])
    
calculate_error(np.median(np.array(preds).reshape(-1, 5), axis=0), ca_val_df['deaths'][-5:].values)

7.329480269431117

In [137]:
data = new_ca_train_df[int(row.begining_index):]
deaths = data['deaths'].values
cases = data['cases'].values
model = armv(k=int(9), bias=True)
model.fit(deaths, [cases], [[14, 15, 16, 17]])
calculate_error(ca_val_df['deaths'][-5:].values, model.predict(len(data), len(data)+4))
# 109	4	4	14	

15.155762216111174